## langchain

In [54]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

In [62]:
# 读取文本

loader = PyMuPDFLoader("..\data\Coggle比赛数据\汽车知识问答\初赛训练数据集.pdf")
PDF_data=loader.load()

In [63]:
# 切分文本
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)

In [64]:

# 利用embedding模型建立向量数据库
persist_directory="db"
model_name = "all-MiniLM-L6-v2"
model_kwargs={"device":"cuda"}
embedding =HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs)
vectordb=Chroma.from_documents(documents=all_splits,embedding=embedding,persist_directory=persist_directory)

loading configuration file config.json from cache at C:\Users\yx140\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\8b3219a92973c328a8e22fadcfa821b5dc75636a\config.json
Model config BertConfig {
  "_name_or_path": "sentence-transformers/all-MiniLM-L6-v2",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file model.safetensors from cache at C:\Users\yx140\.cache\huggingface\hub\models--sentence-transformers--a

In [65]:

from transformers import AutoModelForCausalLM,AutoTokenizer,pipeline
from langchain_huggingface.llms import HuggingFacePipeline
model =AutoModelForCausalLM.from_pretrained('D:\Program Projects\Python Projects\DB-GPT\models\Qwen2-0.5B')
model=model.cuda()

model_path='D:\Program Projects\Python Projects\DB-GPT\models\Qwen2-0.5B'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        torch_dtype='auto'
    ).eval()

pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=100,
        temperature=0.4
    )

return_model = HuggingFacePipeline(pipeline=pipe)

loading configuration file D:\Program Projects\Python Projects\DB-GPT\models\Qwen2-0.5B\config.json
Model config Qwen2Config {
  "_name_or_path": "D:\\Program Projects\\Python Projects\\DB-GPT\\models\\Qwen2-0.5B",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 131072,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

loading weights file D:\Program Projects\Python Projects\DB-GPT\models\Qwen2-0.5B\model.safetensors
Generate config GenerationC

In [66]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=return_model,
    chain_type = "stuff",
    retriever = retriever,
    verbose = True
)

In [67]:
query = "汽车有关知识"
qa.invoke(query)



> Entering new RetrievalQA chain...


d:\ProgramData\miniconda3\envs\big_model\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



> Finished chain.


{'query': '汽车有关知识',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n车辆与前方车辆的距离保持为设定距离、提醒您有碰撞危险等。有些\n情况下，如必要，驾驶辅助系统还会自主刹车。\n警告！\n■驾驶辅助系统对驾驶员来说是一种辅助功能，并不能处理所有情\n\n■如果有，请了解这些图标的含义和相应的处理措施，请参见\n指示灯和警告灯（页码74）。\n■如果无，转至步骤2。\n2 检查是否有任何系统功能下降或受限。\n\n的穿孔，还可以对轮胎重新充气。\n说明！\n□补胎套装仅适用于胎面上存在刺穿的密封轮胎。如果轮胎有较大\n裂缝、裂痕或类似损坏时，无法使用补胎套装密封轮胎。\n补胎套装位于后备厢盖板下方。补胎套装配备有：\n\n■补胎液罐。\n■电动充气泵。\n\nQuestion: 汽车有关知识\nHelpful Answer: The vehicle's distance from the driver's side of the vehicle is set to the maximum distance that can be driven safely. Some situations, such as when the driver is in a hurry, may require the use of the driver's side brake. \n\nThe driver's side brake system will automatically apply the brakes when the driver's side of the vehicle is in collision. \n\nWarning! \n\n■ The driver's side brake system is a safety feature, but it cannot handle all situations. \n\n■ If the